Resources
- https://senbox.atlassian.net/wiki/spaces/SNAP/pages/2499051521/Configure+Python+to+use+the+SNAP-Python+esa_snappy+interface+SNAP+version+10#Installation-of-the-esa_snappy-plugin
- https://github.com/wajuqi/Sentinel-1-preprocessing-using-Snappy/blob/master/s1_preprocessing.py

In [8]:
from esa_snappy import ProductIO
from esa_snappy import ProductIO
from esa_snappy import HashMap
from esa_snappy import GPF

In [ ]:
# path = f'../../data/Sentinel1/sentinel1_central_fl.tif'
path = r"../../data/S1A_IW_GRDH_1SDV_20250320/S1A_IW_GRDH_1SDV_20250320T232815_20250320T232840_058395_0738A3_6166_COG.SAFE/manifest.safe"
sentinel_1 = ProductIO.readProduct(path)
print(sentinel_1)

org.esa.snap.core.datamodel.Product[name=S1A_IW_GRDH_1SDV_20250320T232815_20250320T232840_058395_0738A3_6166_COG]


In [20]:
def do_calibration(source, polarization, pols):
    print('\tCalibration...')
    parameters = HashMap()
    parameters.put('outputSigmaBand', True)
    if polarization == 'DH':
        parameters.put('sourceBands', 'Intensity_HH,Intensity_HV')
    elif polarization == 'DV':
        parameters.put('sourceBands', 'Intensity_VH,Intensity_VV')
    elif polarization == 'SH' or polarization == 'HH':
        parameters.put('sourceBands', 'Intensity_HH')
    elif polarization == 'SV':
        parameters.put('sourceBands', 'Intensity_VV')
    else:
        print("different polarization!")
    parameters.put('selectedPolarisations', pols)
    parameters.put('outputImageScaleInDb', False)
    output = GPF.createProduct("Calibration", parameters, source)
    return output

In [21]:
polarization = 'DV'

if polarization == 'DV':
    pols = 'VH,VV'
elif polarization == 'DH':
    pols = 'HH,HV'
elif polarization == 'SH' or polarization == 'HH':
    pols = 'HH'
elif polarization == 'SV':
    pols = 'VV'
else:
    print("Polarization error!")

calibrated = do_calibration(sentinel_1, polarization, pols)

	Calibration...


In [23]:
ProductIO.writeProduct(calibrated, 'test', 'GeoTIFF')

In [ ]:
sentinel_1.closeIO()

org.esa.snap.core.datamodel.Product(objectRef=0x000001BD2E9A3AF8)

In [26]:
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt

geotiff_path = "test.tif"  # Replace with the actual path
with rasterio.open(geotiff_path) as src:
    geotiff_data = src.read(1)  # Read the first band
    geotiff_extent = rasterio.plot.plotting_extent(src)

fig, ax = plt.subplots()
rasterio.plot.show(geotiff_data, extent=geotiff_extent, ax=ax, cmap='terrain')
ax.set_title("GeoTIFF Image")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.colorbar(ax.images[0], label="Value")
plt.show()

c:\Users\szhan\anaconda3\envs\esa\lib\site-packages\rasterio\__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


MemoryError: Unable to allocate 3.20 GiB for an array with shape (16748, 25653) and data type int64

<Figure size 640x480 with 2 Axes>